In [4]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

client = OpenAI(
  api_key=""
)

# Đọc dữ liệu từ vnexpress_dataset.csv
df = pd.read_csv("./data/vnexpress_dataset.csv")

# Hàm gọi API để tạo tin giả
def generate_fake_news(content):
    prompt = f"""
    Đây là một bản tin thật từ báo chí chính thống:
    [TIN THẬT]: {content}

    Hãy tạo 3 bản tin giả dựa trên nội dung này. Các bản tin giả phải có nội dung hợp lý nhưng không đúng sự thật.
    Không sao chép nguyên văn từ tin thật mà hãy biến đổi câu chữ hoặc thêm thông tin gây hiểu lầm.

    Định dạng đầu ra:
    1. [Bản tin giả 1]
    2. [Bản tin giả 2]
    3. [Bản tin giả 3]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=1024
    )


    if "choices" in response and response["choices"]:
        text = response["choices"][0]["message"]["content"]
        return text.split("\n")  # Tách thành danh sách 3 tin giả
    return [""] * 3  # Trả về rỗng nếu có lỗi

# Tạo danh sách dữ liệu mới
fake_news_list = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating fake news"):
    fake_contents = generate_fake_news(row["Content"])
    for fake_content in fake_contents:
        fake_news_list.append({
            "Title": row["Title"],
            "Link": row["Link"],
            "Views": row["Views"],
            "Comments": row["Comments"],
            "Content": fake_content.strip()
        })

# Lưu vào file vnexpress_fake_dataset.csv
fake_df = pd.DataFrame(fake_news_list)
fake_df.to_csv("vnexpress_fake_dataset.csv", index=False, encoding="utf-8-sig")

print("✅ Đã tạo và lưu tin giả thành công vào vnexpress_fake_dataset.csv")


Generating fake news: 100%|██████████| 1672/1672 [4:07:55<00:00,  8.90s/it]  

✅ Đã tạo và lưu tin giả thành công vào vnexpress_fake_dataset.csv
